In [ ]:
import sys
import numpy as np
from PyQt5.QtWidgets import QWidget, QApplication, QPushButton, QColorDialog, QInputDialog, QHBoxLayout
from PyQt5.QtGui import QPainter, QColor, QPen, QBrush
from PyQt5.QtCore import Qt, QTimer, QCoreApplication

class Game(QWidget):
    def __init__(self, width=50, height=50, gridstep=15):
        super().__init__()

        self.setWindowTitle("Life")
        self.color = QBrush(QColor(100, 0, 0))
        self.delay = 10
        self.height_button = 107
        self.width_button = 100
        self.gridsize = height
        self.grid = np.zeros([height, width], dtype=bool)
        self.gridstep = gridstep
        self.setFixedSize(width * gridstep + self.width_button, height * gridstep)

        self.make_buttons()

        self.show()



    def make_buttons(self):
        width = self.size().width() - self.width_button
        but_start = QPushButton('Start',self)
        but_start.setGeometry(width, 0, self.width_button, self.height_button)
        
        self.is_started = False

        but_stop = QPushButton('Stop', self)
        but_stop.setGeometry(width, self.height_button, self.width_button, self.height_button)
        

        but_next = QPushButton('Next', self)
        but_next.setGeometry(width, 2*self.height_button, self.width_button, self.height_button)
        
        but_figures = QPushButton('Figures',self)
        but_figures.setGeometry(width, 3*self.height_button, self.width_button, self.height_button)
        
        but_settings = QPushButton('Settings', self)
        but_settings.setGeometry(width, 4*self.height_button,  self.width_button, self.height_button)

        but_clear = QPushButton('Clear', self)
        but_clear.setGeometry(width, 5*self.height_button, self.width_button, self.height_button)
        
        but_exit = QPushButton('Exit',self)
        but_exit.setGeometry(width, 6*self.height_button, self.width_button, self.height_button)
        
       
        
        but_start.clicked.connect(self.start)
        but_stop.clicked.connect(self.stop)
        but_next.clicked.connect(self.next)
        but_settings.clicked.connect(self.settings)
        but_clear.clicked.connect(self.clear)
        but_exit.clicked.connect(sys.exit)
        but_figures.clicked.connect(self.figures)

    def paintEvent(self, e):
        qp = QPainter()
        qp.begin(self)
        self.drawLines(qp)
        self.drawCells(qp)
        qp.end()

    def drawCells(self, qp):
        height = self.size().height()
        width = self.size().width() - self.width_button
        step = self.gridstep

        qp.setBrush(self.color)

        for ind1 in range(height//step):
            for ind2 in range(width // step):
                if self.grid[ind2, ind1]:
                    qp.drawRect(ind1 * step, ind2 * step, step, step)

    def drawLines(self, qp):
        pen = QPen(self.color, 2, Qt.SolidLine)
        qp.setPen(pen)
        height = self.size().height()
        width = self.size().width() - self.width_button
        step = self.gridstep
        
        for ind in range(0, height + 1, step):
            qp.drawLine(0, ind, width, ind)
            
        for ind in range(step, width + 1, step):
            qp.drawLine(ind, 0, ind, height)

    def mousePressEvent(self, event):
        if not self.is_started:
            x, y = event.x() // self.gridstep, event.y() // self.gridstep
            
            self.grid[y, x] = not self.grid[y, x]
            self.update()
            self.press_update = (y, x)


    def mouseMoveEvent(self, event):
        if not self.is_started:
            x, y = event.x() // self.gridstep, event.y() // self.gridstep
            if self.press_update != (y, x):
                if x >= 0 and y >= 0 and event.x() < self.width() - self.width_button and event.y() < self.height():
                    self.grid[y, x] = not self.grid[y, x]
                    self.update()
                    self.press_update = (y, x)

    def mouseReleaseEvent(self, event):
        self.press_update = (-1, -1)


    def start(self):
        self.is_started = True
        self.go()

    def stop(self):
        self.is_started = False

    def next(self):
        self.is_started = False
        self.go()

    def settings(self):
        self.sett = Settings(self)
        
    def figures(self):
        self.fig = Figures(self)

    def clear(self):
        self.is_started = False
        self.grid = np.zeros([(self.size().height()) // self.gridstep,
                              (self.size().width() - self.width_button) // self.gridstep], dtype=bool)
        self.update()


    def go(self):
        if not self.update_grid():
            self.is_started = False
            return
        self.update()
        if self.is_started:
            QTimer.singleShot(self.delay, self.go)
        else:
            return

    def update_grid(self):
        new_grid = np.copy(self.grid)

        height = self.size().height()
        width = self.size().width() - self.width_button
        step = self.gridstep
        gheight = height // step
        gwidth = width // step

        for i in range(gheight):
            for j in range(gwidth):
                neib = 0
                for k in range(-1, 2):
                    for n in range(-1, 2):
                        x = j + n if j + n < gheight else 0
                        y = i + k if i + k < gwidth else 0
                        if self.grid[y, x]:
                            neib += 1
                if self.grid[i, j]:
                    neib -= 1
                    if neib != 2 and neib != 3:
                        new_grid[i, j] = False
                else:
                    if neib == 3:
                        new_grid[i, j] = True

        if (self.grid == new_grid).all():
            return False

        self.grid = new_grid
        return True
    
class Figures(QWidget):
    def __init__(self, parent):
        super().__init__()
        self.parent = parent
        self.setWindowTitle("Figures")

        self.height_button = 300
        self.width_button = 300
        self.setFixedSize(self.height_button*2, self.width_button*2)

        self.make_buttons()
        self.newcolor = parent.color
        self.delay = parent.delay

        self.show()

    def make_buttons(self):
        but_barge = QPushButton('Barge', self)
        but_barge.setGeometry(0, 0, self.width_button, self.height_button)

        but_snake = QPushButton('Snake', self)
        but_snake.setGeometry(self.width_button, 0, self.width_button, self.height_button)

        but_ship = QPushButton('Ship', self)
        but_ship.setGeometry(0, self.height_button, self.width_button, self.height_button)

        but_box = QPushButton('Box', self)
        but_box.setGeometry(self.width_button, self.height_button, self.width_button, self.height_button)

        but_barge.clicked.connect(self.set_barge)
        but_snake.clicked.connect(self.set_snake)
        but_ship.clicked.connect(self.set_ship)
        but_box.clicked.connect(self.set_box)

    def set_barge(self):
        if not self.parent.is_started:
            for ind in range(int(self.parent.gridsize/2) - 2,int(self.parent.gridsize/2) + 1):
                self.parent.grid[ind,ind] = 1
                self.parent.grid[ind+1,ind-1] = 1
    
        self.hide()

    def set_snake(self):
        if not self.parent.is_started:
            for ind in range(int(self.parent.gridsize/2) - 2,int(self.parent.gridsize/2) + 1):
                self.parent.grid[ind,int(self.parent.gridsize/2) - 2] = 1
                self.parent.grid[ind+1,int(self.parent.gridsize/2) - 3] = 1
            
        self.hide()
        
    def set_ship(self):
        size = int(self.parent.gridsize/2)
        self.parent.grid[size-1,size-1], self.parent.grid[size,size-1], self.parent.grid[size-1,size] = 1,1,1
        self.parent.grid[size+1,size+1], self.parent.grid[size,size+1], self.parent.grid[size+1,size] = 1,1,1
        
        self.hide()
    
    def set_box(self):
        size = int(self.parent.gridsize/2)
        for ind in range(size - 4, size):
            self.parent.grid[ind, size] = 1
            self.parent.grid[ind, size + 5] = 1
            self.parent.grid[size - 5, ind + 5] = 1
            self.parent.grid[size, ind + 5] = 1
            
        self.parent.grid[size+2,size+1],self.parent.grid[size+3,size+2],self.parent.grid[size+2,size+2], \
            self.parent.grid[size+3,size+1]=1,1,1,1
        self.parent.grid[size-2,size+7],self.parent.grid[size-2,size+8],self.parent.grid[size-1,size+7], \
            self.parent.grid[size-1,size+8]=1,1,1,1
        self.parent.grid[size-7,size+3],self.parent.grid[size-7,size+4],self.parent.grid[size-8,size+3], \
            self.parent.grid[size-8,size+4]=1,1,1,1
        self.parent.grid[size-3,size-2],self.parent.grid[size-4,size-2],self.parent.grid[size-3,size-3], \
            self.parent.grid[size-4,size-3]=1,1,1,1
        
        self.parent.grid[size-2,size+2],self.parent.grid[size-4,size+3],self.parent.grid[size-3,size+1] \
            = 1,1,1
            
        self.hide()

class Settings(QWidget):
    def __init__(self, parent):
        super().__init__()
        self.parent = parent
        self.setWindowTitle("Settings")

        self.height_button = 300
        self.width_button = 300
        self.setFixedSize(self.height_button*2, self.width_button*2)

        self.make_buttons()
        self.newcolor = parent.color
        self.delay = parent.delay

        self.show()
    
    def make_buttons(self):
        but_color = QPushButton('Color', self)
        but_color.setGeometry(0, 0, self.width_button, self.height_button)

        but_delay = QPushButton('Delay', self)
        but_delay.setGeometry(self.width_button, 0, self.width_button, self.height_button)

        but_cancel = QPushButton('Cancel', self)
        but_cancel.setGeometry(0, self.height_button, self.width_button, self.height_button)

        but_ok = QPushButton('Ok', self)
        but_ok.setGeometry(self.width_button, self.height_button, self.width_button, self.height_button)

        but_color.clicked.connect(self.set_color)
        but_delay.clicked.connect(self.set_delay)
        but_cancel.clicked.connect(self.cancel)
        but_ok.clicked.connect(self.ok)

    
    def set_color(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.newcolor = color

    def set_delay(self):
        ok = False
        delay, ok = QInputDialog.getInt(self, "Set delay", "New delay:", value=1, min=1)
        if ok:
            self.delay = delay


    def cancel(self):
        self.hide()

    def ok(self):
        self.parent.color = self.newcolor
        self.parent.delay = self.delay
        self.hide()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = Game()
    sys.exit(app.exec_())